# 1. Imports

In [1]:
from sklearn.decomposition import PCA
import matplotlib as mpl 
import numpy as np
import os
import pandas as pd
import pickle

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.metrics import f1_score
from imblearn.over_sampling import RandomOverSampler
from sklearn.preprocessing import StandardScaler


from codes.utils import stratified_train_test_group_kfold
from codes.utils import model_test_classification

from codes.classification_codes import gpc_gridsearch



0.10.1


# 2. Data

In [2]:
with open("Data/MoS2_Analysis_Data_trained2", "rb") as fp:   # Unpickling
    MoS2_Proj1_Class_Data = pickle.load(fp)

#features = MoS2_ImageNet_100_data[0]
df = pd.DataFrame(MoS2_Proj1_Class_Data)
print(df.keys())
T_dict= {900:0, 950:1, 1000:2}
T_target = np.array([T_dict[item] for item in df['T']])

features = np.array(list(df['ImageNet']))
sampleId = np.array(list(df['sampleId']))

X = features
Y = np.array(T_target)
groups = np.array(sampleId)


train_val_groups, train_val_X, train_val_Y, test_X, test_Y = stratified_train_test_group_kfold(X, Y, groups, n_splits=10, test_fold=0)

train_val_Y = train_val_Y.flatten()
test_Y = test_Y.flatten()

print(train_val_X.shape)
print(train_val_Y.shape)
print(test_X.shape)
print(test_Y.shape)


oversample = RandomOverSampler(sampling_strategy='not majority')

scaler = StandardScaler().fit(train_val_X)
train_val_X = scaler.transform(train_val_X)
test_X = scaler.transform(test_X)

Index(['sampleId', 'sampleLabel', 'image', 'T', 'ImageNet', 'MicroNet'], dtype='object')


In [3]:
! mkdir -p 'Model/Class/TrainedImageNet/Up/'

# 3. Runing Cross-Validation

In [5]:

def mul_range(start, stop, multiple):
    value = start
    while value < stop: # using < for consistency with range
        yield value
        value *= multiple
N_restarts_optimizer = [i for i in range(5)]
Max_iter_predict = [i for i in range(100, 501, 100)]
Multi_class = ['one_vs_rest', 'one_vs_one']


def cross_10_folds_gpc(train_val_X, train_val_Y):
    
    best_train = []
    best_val = []
    best_variables = []
    for fold in range(10):
        
        group, train_X, train_Y, val_X, val_Y = stratified_train_test_group_kfold(train_val_X, train_val_Y, train_val_groups, n_splits=10, test_fold=fold)
        
        model_path = f'classification/ImageNet/aug3/GPC_{fold}_model.sav'
        
        train_X, train_Y = oversample.fit_resample(train_X, train_Y)
        
        variables_best, performance_best = gpc_gridsearch(train_X, train_Y, val_X, val_Y, N_restarts_optimizer, Max_iter_predict, Multi_class, fold, model_path)
        best_train.append(performance_best['accuracy_train'])
        best_val.append(performance_best['accuracy_val'])
        best_variables.append(variables_best)
        print(f'fold: {fold} done!')
    return best_train, best_val, best_variables


best_train, best_val, best_variables = cross_10_folds_gpc(train_val_X, train_val_Y)
print(best_train)
print(best_val)
print(f'train_acc_mean: {np.mean(best_train) :.3f}, std: {np.std(best_train) :.3f}, val_acc_mean: {np.mean(best_val):.3f}, std: {np.std(best_val):.3f}')

<class 'generator'>
best_train_acc: 1.0, best_val_acc: 0.5833333333333334
best_train_acc: 1.0, best_val_acc: 0.625
fold: 0 done!
<class 'generator'>
best_train_acc: 1.0, best_val_acc: 0.6666666666666666
best_train_acc: 1.0, best_val_acc: 0.7083333333333334
fold: 1 done!
<class 'generator'>
best_train_acc: 1.0, best_val_acc: 0.5833333333333334
best_train_acc: 1.0, best_val_acc: 0.625
fold: 2 done!
<class 'generator'>
best_train_acc: 1.0, best_val_acc: 0.5416666666666666
best_train_acc: 1.0, best_val_acc: 0.5833333333333334
fold: 3 done!
<class 'generator'>
best_train_acc: 1.0, best_val_acc: 0.75
fold: 4 done!
<class 'generator'>
best_train_acc: 1.0, best_val_acc: 0.43478260869565216
fold: 5 done!
<class 'generator'>
best_train_acc: 1.0, best_val_acc: 0.5652173913043478
best_train_acc: 1.0, best_val_acc: 0.6086956521739131
fold: 6 done!
<class 'generator'>
best_train_acc: 1.0, best_val_acc: 0.5652173913043478
best_train_acc: 1.0, best_val_acc: 0.6086956521739131
fold: 7 done!
<class 'gen

# 4. Model Testing

In [6]:
best_fold = 4

model_path = f'classification/ImageNet/aug3/GPC_{best_fold}_model.sav'
# Fitting different train in the same hyperparameters
best_test, confusion_matrix_test = model_test_classification(train_val_X, train_val_Y, train_val_groups,test_X, test_Y, best_fold, model_path)



print(best_test)
print(f'test_acc_mean: {np.mean(best_test) :.3f}, std: {np.std(best_test) :.3f}')

print('5th confusion matrix: ', [list(item) for item in confusion_matrix_test[4]])

<class 'generator'>
<class 'generator'>
<class 'generator'>
<class 'generator'>
<class 'generator'>
<class 'generator'>
<class 'generator'>
<class 'generator'>
<class 'generator'>
<class 'generator'>
[0.6296296296296297, 0.6666666666666666, 0.6666666666666666, 0.6296296296296297, 0.6666666666666666, 0.6666666666666666, 0.6666666666666666, 0.7037037037037037, 0.6296296296296297, 0.6296296296296297]
test_acc_mean: 0.656, std: 0.024
5th confusion matrix:  [[1, 2, 0], [1, 8, 4], [0, 2, 9]]


In [7]:
cm_list = []
for cm in confusion_matrix_test:
    cm_list.append([list(item) for item in cm])
#print(cm_list)

print(len(cm_list))
print([list(item) for item in np.mean(cm_list, axis=0)])

10
[[1.0, 1.9, 0.1], [1.3, 7.8, 3.9], [0.0, 2.1, 8.9]]


In [3]:
best_fold = 4
PATH = f'Model/Class/TrainedImageNet/Up/GPC_{best_fold}_model.sav'

#PATH = os.path.join('Models', model_path)
loaded_model = pickle.load(open(PATH, 'rb'))
loaded_model.get_params()

{'copy_X_train': True,
 'kernel': None,
 'max_iter_predict': 100,
 'multi_class': 'one_vs_rest',
 'n_jobs': None,
 'n_restarts_optimizer': 0,
 'optimizer': 'fmin_l_bfgs_b',
 'random_state': None,
 'warm_start': False}